In [2]:
import time
import datetime
import pandas as pd
import os
import warnings
import json
import csv
import boto3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selectolax.parser import HTMLParser
from urllib.parse import unquote
from botocore.client import Config

In [1]:
# Структура извлекаемых данных
columns_avito = [        'url_offer'        # cсылка на объявление
                        , 'id_offer'        # ID объявления
                        , 'seller'          # продавец (cобственник/агентство)
                        , 'date_offer'      # дата публикации объявления
                        , 'rooms_offer'     # количество комнат в квартире
                        , 'rooms_type'      #тип комнат в квартире   
                        , 'floors_house'    # этажность дома
                        , 'floor_offer'     # этаж квартиры
                        , 'type_offer'      # тип объявления (новостройка/вторичное жилье)
                        , 'height_offer'    # высота потолков, м
                        , 'square_total_offer'      # площадь квартиры общая, м2
                        , 'square_rooms_offer'      # площадь квартиры жилая, м2 
                        , 'square_kitchen_offer'    # площадь кухни, м2
                        , 'price_offer'             # цена квартиры в объявлении, руб.'
                        , 'renovation_offer'        # ремонт квартиры
                        , 'address_offer'           # адрес квартиры в формате: город, улица, дом
                        , 'latitude'            # координаты дома
                        , 'longitude'           # координаты дома
                        , 'metro_name1'         # ближайшая станция 1
                        , 'metro_name2'         # ближайшая станция 2
                        , 'metro_name3'         # ближайшая станция 3
                        , 'time_to_metro1'      # время ходьбы до ближайшей станции метро 1
                        , 'time_to_metro2'      # время ходьбы до ближайшей станции метро 2
                        , 'time_to_metro3'      # время ходьбы до ближайшей станции метро 3
                        , 'distance_to_metro1'  # расстояние до ближайшей станции метро 1, м
                        , 'distance_to_metro2'  # расстояние до ближайшей станции метро 2, м
                        , 'distance_to_metro3'  # расстояние до ближайшей станции метро 3, м
                        , 'sdelka_offer'        # способ продажи (наличие обременения на квартиру)     
                        , 'ipoteka_offer'       # вид сделки (возможна ли ипотека)
                        , 'warm_floor'          # наличие тёплого пола       
                        , 'furniture'           # наличие мебели       
                        , 'home_appliances'     # наличие бытовой техники
                        , 'garderob'            # наличие гардеробной
                        , 'panoram_window'      # наличие панорамных окон
                        , 'bathroom'            # тип санузла (совмещенный/раздельный)
                        , 'balcony_type'        # тип балкона
                        , 'window_view'         # вид из окна (на улицу/во двор)
                        , 'built_year_offer'    # год постройки дома
                        , 'type_house_offer'    # тип дома
                        , 'autopark_offer'      # тип автопарковки
                        , 'rubbish_chute_offer' # наличие мусоропровода
                        , 'security_offer'      # консьерж
                        , 'gas_offer'           # газ
                        , 'closed_yard'         # закрытая территория двора
                        , 'child_playground'    # наличие десткой площадки
                        , 'sport_playground'    # наличие спортивной площадки
                        , 'lift_pass'           # количество пассажирских лифтов
                        , 'lift_gruz'           # количество грузовых лифтов
                        , 'description_offer'   # описание квартиры от продавца
                        , 'photo_list_offer'    # список ссылок на фотографии
                        , 'sobstvenn_count'     # количество собственников
                        , 'date_sobstvenn_change' # последняя смена собственников
                        , 'ogranicheniya'       # ограничения и обременения
                        , 'sovpadenie_kharakteristik' # совпадение характеристик
                        , 'developer_offer'     # застройщик
                        , 'otdelka'             # отделка (для новостройки)
                        , 'novonostroyka_name'  # название ЖК/новостройки
                        , 'uchastie'            # тип участия
                        , 'built_kvartal_offer' # срок сдачи (для новостроек)
                       ]

In [18]:
def get_html_by_selenium(url_page):
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; WOW64; rv:45.0) Gecko/20100101 Firefox/45.0")
    options.add_argument("--disable-blink-features=AutomationControlled")
    s = Service()
    driver = webdriver.Chrome(service = s, options = options)
    try:
        driver.get(url = url_page)
        time.sleep(2)
        driver.implicitly_wait(10)
        html = driver.page_source
        time.sleep(2)
    except Exception as ex:
        print(ex)
    finally:
        driver.close()
        driver.quit()
    return html       

In [4]:
#Функция получает html разметку карточки квартиры, находит json 
def transform_html_to_json(html:str, path:str = 'C:\\Users\\fnona\\OneDrive\\Документы\\Pars_Avito\\Json'):
    tree = HTMLParser(html)
    scripts = tree.css('script')
    for script in scripts:
        if 'window.__initialData' in script.text():
            json_data = script.text()
    json_data = json_data.split(';')[0].split('=')[-1].strip()
    json_data = unquote(json_data)
    json_data = json_data[1:-1]
    if type(json_data) == dict:
        data = json_data
    elif type(json_data) == str:
        data = json.loads(json_data)   
    os.chdir(path)
    i = len(os.listdir())
    if len(str(i)) == 1:
        i = '0000' + str(i)
    elif len(str(i)) == 2:
        i = '000' + str(i)
    elif len(str(i)) == 3:
        i = '00' + str(i) 
    elif len(str(i)) == 4:
        i = '0' + str(i) 
    with open(f'{i}_data.json', 'w', encoding= 'utf-8') as file:
        json.dump(data, file, ensure_ascii= False)
    return data

In [5]:
#Функция, которая находит основной узел (ключ) в json, в котором собраны основные сведения по объекту
def main_avito_key(offer_json:dict):
    for key in offer_json:
        if 'avito' in key:
            return key

In [6]:
#Функция получает словарь (json с данными) и список упорядоченных ключей
#Функция проверяет наличие вложенных словарей для поиска итогового значения
#возвращает найденное значение, или None, если любой ключ из списка будет отсутствовать
def get_value(offer: dict, keys_list: list):
    if len(keys_list) == 1:
        if keys_list[0] in offer:
            return offer[keys_list[0]]
        else:
            return None
    else:
        if keys_list[0] in offer:
            return get_value(offer[keys_list[0]], keys_list[1:])
        else:
            return None

In [7]:
#Функция преобразует строку с датой объявления в авито в формат datetime: "%d-%m-%Y"
def str_to_date(str_date):
    date_dict = {'января': '01', 'февраля':'02','марта':'03',
                 'апреля':'04','мая':'05','июня':'06',
                 'июля':'07','августа':'08','сентября':'09',
                 'октября':'10','ноября':'11','декабря':'12'}
    if 'сегодня' in str_date:
        return datetime.date.today()
    elif 'вчера' in str_date:
        return datetime.date.today() - datetime.timedelta(1)
    else:
        str_date = str_date.split(' ')
        str_day = str_date[0]
        str_month = date_dict[str_date[1]]
        return datetime.datetime.strptime(f'{str_day}-{str_month}-2025', "%d-%m-%Y").date()

In [8]:
def recalc_metro_dist(distance_to_metro):
    distance_dict = {'м': 1, 'км': 1000}
    if type(distance_to_metro) == str:
        distance_to_metro = distance_to_metro.replace(',', '.')
        distance_to_metro = distance_to_metro.split(' ')
        distance_to_metro = float(distance_to_metro[1]) * distance_dict[distance_to_metro[2]]
        return distance_to_metro

In [9]:
def space_red(text):
    if text == None:
        return None
    text = text.replace('\xa0', ' ')
    return text

In [10]:
#Функция получает json и извлекает данные в датафрейм
def pars_avito_card(offer:dict):
    df_page = pd.DataFrame(columns = columns_avito)
    avito_main_knot = main_avito_key(offer)
# cсылка на объявление
    url_keys = [avito_main_knot, 'buyerItem','paramsDto', 'itemUrl']
    url_offer = get_value(offer, url_keys)
    if url_offer == None:
        url_keys = [avito_main_knot, 'buyerItem', 'contactBarInfo', 'contacts', 'list']
        list_cont = get_value(offer, url_keys)
        if list_cont == None:
            url_offer = 'Отсутствует ссылка'
        elif len(list_cont) >= 2:
            url_keys = ['value', 'itemUrl']
            url_offer = get_value(list_cont[1], url_keys)
        else:
            url_offer = 'Отсутствует ссылка'
    if url_offer != 'Отсутствует ссылка':
        url_offer = 'https://www.avito.ru' + url_offer

# ID объявления
    id_keys = [avito_main_knot, 'buyerItem', 'item', 'id']
    id_offer = get_value(offer, id_keys)

# дата публикации объявления
    date_offer_keys = [avito_main_knot, 'buyerItem', 'item', 'sortFormatedDate']
    date_offer = get_value(offer, date_offer_keys)

# Продавец (cобственник/агентство)
    seller_keys = [avito_main_knot, 'buyerItem', 'contactBarInfo', 'publicProfileInfo', 'sellerName']
    seller = get_value(offer, seller_keys)

# дата публикации объявления
    date_offer_keys = [avito_main_knot, 'buyerItem', 'item', 'sortFormatedDate']
    date_offer = get_value(offer, date_offer_keys)
    date_offer = str_to_date(date_offer)

# адрес квартиры в формате: город, улица, дом
    address_offer_keys = [avito_main_knot, 'buyerItem', 'item', 'address']
    address_offer = get_value(offer, address_offer_keys)

# координаты дома
    coords_offer_keys = [avito_main_knot, 'buyerItem', 'item', 'geo', 'coords']
    coords_offer = get_value(offer, coords_offer_keys)
    if coords_offer == None:
        latitude, longitude = None, None
    else:    
        latitude = coords_offer.get('lat', None)
        longitude = coords_offer.get('lng', None)

    ga_offer_keys = [avito_main_knot, 'buyerItem', 'ga']
    list_ga = get_value(offer, ga_offer_keys)
    if list_ga == None:
        rooms_offer, floors_house, floor_offer, type_offer = None, None, None, None
        square_total_offer, square_rooms_offer, square_kitchen_offer = None, None, None
        price_offer, balcony_type, built_year_offer, type_house_offer = None, None, None, None
        bathroom, lift_pass, lift_gruz, developer_offer, otdelka = None, None, None, None, None
    elif len(list_ga) >= 2:
    # количество комнат в квартире
        rooms_offer = get_value(list_ga[1], ['rooms'])
        if type(rooms_offer) == str and rooms_offer != 'Студия':
            rooms_offer = int(rooms_offer)
    # этажность дома
        floors_house = get_value(list_ga[1], ['floors_count'])
    # этаж квартиры
        floor_offer = get_value(list_ga[1], ['floor'])
    # тип объявления (новостройка/вторичное жилье)
        type_offer  = get_value(list_ga[1], ['type'])
    # площадь квартиры общая, м2
        square_total_offer = get_value(list_ga[1], ['area'])
        if type(square_total_offer) == str:
            square_total_offer = float(square_total_offer.split()[0])
    # площадь квартиры жилая, м2     
        square_rooms_offer = get_value(list_ga[1], ['area_live'])
        if type(square_rooms_offer) == str:
            square_rooms_offer = float(square_rooms_offer.split()[0])
    # площадь кухни, м2
        square_kitchen_offer = get_value(list_ga[1], ['area_kitchen'])
        if type(square_kitchen_offer) == str:
            square_kitchen_offer = float(square_kitchen_offer.split()[0])
    # цена квартиры в объявлении, руб.
        price_offer = get_value(list_ga[1], ['itemPrice'])
    # тип балкона:
        balcony_type = get_value(list_ga[1], ['balkon_ili_lodzhiya_multi'])
    # год постройки дома
        built_year_offer = get_value(list_ga[1], ['god_postroiki'])
    # тип дома
        type_house_offer = get_value(list_ga[1], ['house_type'])
    # тип санузла (совмещенный/раздельный)
        bathroom = get_value(list_ga[1], ['sanuzel_multiple'])
    # количество пассажирских лифтов
        lift_pass = get_value(list_ga[1], ['passazhirskii_lift'])
        if type(lift_pass) == str:
            lift_pass = int(lift_pass)
    # количество грузовых лифтов
        lift_gruz = get_value(list_ga[1], ['gruzovoi_lift'])
        if type(lift_gruz) == str:
            lift_gruz = int(lift_gruz)
    # застройщик
        developer_offer = get_value(list_ga[1], ['ofitsialnyy_zastroyshchik'])
    # отделка (для новостройки)
        otdelka = get_value(list_ga[1], ['otdelka'])

    items_offer_keys = [avito_main_knot, 'buyerItem', 'paramsBlock', 'items']
    list_items = get_value(offer, items_offer_keys)
    rooms_type, height_offer, window_view, renovation_offer, sdelka_offer = None, None, None, None, None
    ipoteka_offer, warm_floor, furniture, home_appliances, garderob = None, None, None, None, None
    panoram_window = None
    if list_items != None:
        if len(list_items) >= 1:
            for item in list_items:
                if item['title'] == 'Тип комнат':
                # тип комнат в квартире
                    rooms_type = item.get('description', None)    
                elif item['title'] == 'Высота потолков':
                # высота потолков, м
                    height_offer = item.get('description', None)
                    if type(height_offer) == str:
                        height_offer = float(height_offer.split('\xa0')[0])
                elif item['title'] == 'Окна':
                # вид из окна (на улицу/во двор)
                    window_view = item.get('description', None)
                elif item['title'] == 'Ремонт':
                # ремонт квартиры
                    renovation_offer = item.get('description', None)
                elif item['title'] == 'Способ продажи':
                # способ продажи (наличие обременения на квартиру)
                    sdelka_offer = item.get('description', None)     
                # вид сделки (возможна ли ипотека)
                elif item['title'] == 'Вид сделки':
                    ipoteka_offer = item.get('description', None)
                # наличие тёплого пола
                elif item['title'] == 'Тёплый пол':
                    warm_floor = item.get('description', None)
                # наличие мебели
                elif item['title'] == 'Мебель':
                    furniture = item.get('description', None)
                # наличие бытовой техники
                elif item['title'] == 'Техника':
                    home_appliances = item.get('description', None)
                # дополнительные опции (гардеробная, панорамные окна и т.д.)
                elif item['title'] == 'Дополнительно':
                    additional_info = item.get('description', None)
                    garderob = 'гардероб' in additional_info
                    panoram_window = 'панорамные окна' in additional_info

# описание квартиры от продавца
    description_offer_keys = [avito_main_knot, 'buyerItem', 'seoDescription']
    description_offer = get_value(offer, description_offer_keys)
    if description_offer != None:
        description_offer = description_offer.replace('\xa0', ' ')

# список ссылок на фотографии
    photo_offer_keys = [avito_main_knot, 'buyerItem', 'galleryInfo', 'media']
    photo_list = get_value(offer, photo_offer_keys)
    photo_list_offer = []
    if photo_list != None:
        for photo in photo_list:
            photo_list_offer.append(get_value(photo, ['urls','1280x960']))

    house_params_keys = [avito_main_knot, 'buyerItem', 'item', 'houseParams', 'data', 'items']
    house_params = get_value(offer, house_params_keys)
    rubbish_chute_offer, security_offer, gas_offer, closed_yard, child_playground = None, None, None, None, None
    sport_playground, autopark_offer, novonostroyka_name, uchastie, built_kvartal_offer = None, None, None, None, None
    if house_params != None:
        for param in house_params:
            if param['title'] == 'В доме':
                v_dome = param.get('description', None)
                # наличие мусоропровода
                rubbish_chute_offer = 'мусоропровод' in v_dome
                # консьерж
                security_offer = 'консьерж' in v_dome
                # газ
                gas_offer = 'газ' in v_dome
            elif param['title'] == 'Двор':
                dvor_offer = param.get('description', None)
                # закрытая территория двора
                closed_yard = 'закрытая территория' in dvor_offer
                # наличие десткой площадки
                child_playground = 'детская площадка' in dvor_offer
                # наличие спортивной площадки
                sport_playground = 'спортивная площадка' in dvor_offer
            elif param['title'] == 'Парковка':
                # тип автопарковки
                autopark_offer = param.get('description', None)
            elif param['title'] == 'Название новостройки':
                # название ЖК/новостройки
                novonostroyka_name = param.get('description', None)
            elif param['title'] == 'Тип участия':
                # тип участия
                uchastie = param.get('description', None)
            elif param['title'] == 'Срок сдачи':
                # срок сдачи (для новостроек)
                built_kvartal_offer = param.get('description', None)

    metro_offer_keys = [avito_main_knot, 'buyerItem', 'item', 'geo', 'references']
    metro_list = get_value(offer, metro_offer_keys)
    metro_name1, metro_name2, metro_name3 = None, None, None
    time_to_metro1, time_to_metro2, time_to_metro3 = None, None, None
    distance_to_metro1, distance_to_metro2, distance_to_metro3 = None, None, None
    if metro_list != None:
        # характеристики метро: парсятся данные до 3 ближайших станций метро
        # ближайшая станция
        metro_name1 = metro_list[0].get('content', None)
        metro_name2 = metro_list[1].get('content', None)
        metro_name3 = metro_list[2].get('content', None)
        # время ходьбы до метро
        time_to_metro1 = metro_list[0]['afterWithIcon'].get('text', None)
        time_to_metro2 = metro_list[1]['afterWithIcon'].get('text', None)
        time_to_metro3 = metro_list[2]['afterWithIcon'].get('text', None)
        # расстояние до метро, м
        distance_to_metro1 = metro_list[0].get('after', None)
        distance_to_metro1 = recalc_metro_dist(distance_to_metro1)
        distance_to_metro2 = metro_list[1].get('after', None)
        distance_to_metro2 = recalc_metro_dist(distance_to_metro2)
        distance_to_metro3 = metro_list[2].get('after', None)
        distance_to_metro3 = recalc_metro_dist(distance_to_metro3)

    # блок проверки по Росреестру
    rosreestr_offer_keys = [avito_main_knot, 'buyerItem', 'domotekaReportTeaser', 'infoList']
    rosreestr_list = get_value(offer, rosreestr_offer_keys)
    sobstvenn_count, date_sobstvenn_change, ogranicheniya, sovpadenie_kharakteristik = None, None, None, None
    if rosreestr_list != None:
        # количество собственников
        sobstvenn_count = rosreestr_list[0].get('text', None)
        sobstvenn_count = int(sobstvenn_count.split('\xa0собственн')[0])
        # последняя смена собственников
        date_sobstvenn_change = rosreestr_list[1].get('text', None)
        # ограничения и обременения
        ogranicheniya = space_red(rosreestr_list[2].get('text', None))
        # совпадение характеристик
        sovpadenie_kharakteristik = space_red(rosreestr_list[3].get('text', None))
        
    df_page.loc[len(df_page)] = [url_offer, id_offer, seller, date_offer, rooms_offer,
rooms_type, floors_house, floor_offer, type_offer, height_offer, 
square_total_offer, square_rooms_offer, square_kitchen_offer, price_offer, renovation_offer,
address_offer, latitude, longitude, metro_name1, metro_name2,
metro_name3, time_to_metro1, time_to_metro2, time_to_metro3, distance_to_metro1,
distance_to_metro2, distance_to_metro3, sdelka_offer, ipoteka_offer, warm_floor,
furniture, home_appliances, garderob, panoram_window, bathroom, 
balcony_type, window_view, built_year_offer, type_house_offer, autopark_offer,
rubbish_chute_offer, security_offer, gas_offer, closed_yard, child_playground,
sport_playground, lift_pass, lift_gruz, description_offer, photo_list_offer,
sobstvenn_count, date_sobstvenn_change, ogranicheniya, sovpadenie_kharakteristik, 
developer_offer, otdelka, novonostroyka_name, uchastie, built_kvartal_offer]
    os.chdir('C:\\Users\\fnona\\OneDrive\\Документы\\Pars_Avito\\csv')
    df_page.to_csv(str(id_offer)+'_Avito_'+ str(datetime.date.today().isoformat()) + '.csv', encoding = 'utf-8') 
    return df_page

In [11]:
#Функция получает на вход список ссылок на карточки объектов
def main_pars_avito(start_url:str, pages_count: int):
    url_list_to_pars = get_list_url_to_pars(start_url, pages_count)
    df_result_avito = pd.DataFrame(columns = columns_avito)
    for url in url_list_to_pars:
        html = get_html_by_selenium(url)
        json_data = transform_html_to_json(html)
        offer_data = pars_avito_card(json_data)
        df_result_avito = pd.concat([df_result_avito, offer_data], axis = 0, join = 'outer')

In [12]:
def get_list_url_to_pars(start_url:str, pages_count: int):
    keys_list_of_items = ['data', 'catalog', 'items']
    list_url_to_pars = []
    for i in range(1, pages_count + 1):
        if i > 1:
            start_url = start_url + '&p=' + str(i)
        html_start_page = get_html_by_selenium('view-source:' + start_url)
        index1 = html_start_page.find('window.__initialData__')
        index2 = html_start_page.find('window.__mfe__')
        html_start_page = html_start_page[index1:index2]
        html_start_page = html_start_page.replace('&amp;quot;', '\"')
        html_start_page = html_start_page.replace('&nbsp;', ' ')
        html_start_page = html_start_page.split('=')[1].split(';')[0].strip()
        html_start_page = unquote(html_start_page)
        html_start_page = html_start_page[1:-1]
        if type(html_start_page) == dict:
            data = html_start_page
        elif type(html_start_page) == str:
            data = json.loads(html_start_page)
        list_of_items = get_value(data, keys_list_of_items)
        for i in range(len(list_of_items)):
            url_item = list_of_items[i].get('urlPath', None)
            if url_item != None:
                list_url_to_pars.append('avito.ru' + url_item)
    with open("list_url_to_pars.txt" , 'w', encoding= 'utf-8') as file:
        file.write(list_url_to_pars)
    return list_url_to_pars

In [ ]:
def s3_load(file_to_load, name):
    s3 = boto3.client(service_name = 's3',
                endpoint_url = 'https://storage.yandexcloud.net',    
                aws_access_key_id = aws_access_key_id
                aws_secret_access_key = aws_secret_access_key
                config=Config())
    with open(file_to_load, "rb") as file:
        s3.upload_fileobj(file, 'rvaitables', name)

In [14]:
date_dict = {'января': '01', 'февраля':'02','марта':'03',
                 'апреля':'04','мая':'05','июня':'06',
                 'июля':'07','августа':'08','сентября':'09',
                 'октября':'10','ноября':'11','декабря':'12'}

In [15]:
start_url = 'view-source:https://www.avito.ru/moskva/kvartiry/prodam/vtorichka-ASgBAgICAkSSA8YQ5geMUg?context=H4sIAAAAAAAA_wEjANz_YToxOntzOjg6ImZyb21QYWdlIjtzOjc6ImNhdGFsb2ciO312FITcIwAAAA'